In [1]:
import pandas as pd
import numpy as np
from lightgbm import LGBMClassifier, LGBMRegressor
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from xgboost import XGBClassifier, XGBRegressor
from catboost import CatBoostClassifier, CatBoostRegressor
from sklearn.linear_model import LogisticRegression, Ridge
import lightgbm
from scipy.optimize import minimize
from sklearn.preprocessing import StandardScaler


In [2]:
train = pd.read_csv('../Data/train.csv')
test = pd.read_csv('../Data/pre_contest_test.csv')

In [3]:
train_targets = train['label']
train.drop("label", axis=1, inplace=True)

In [32]:
test_preds = np.zeros((len(test), 6))
cv = 0
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
final_train = []
final_test = []
for fold, (train_idx, valid_idx) in enumerate(kf.split(train, train_targets)):
    print("-"*10 + ">" + f"Fold {fold+1}" + "<" + "-"*10)
    
    X_train, X_valid = train.iloc[train_idx], train.iloc[valid_idx]
    y_train, y_valid = train_targets.iloc[train_idx], train_targets.iloc[valid_idx]
    
    
    ctb =CatBoostClassifier(n_estimators=10000, 
                         early_stopping_rounds=100, 
                         **{'depth': 3, 'subsample': 0.9037951675853159, 'l2_leaf_reg': 4.085968446512874, 
                            'learning_rate': 0.33153188532829714,"thread_count":-1,'loss_function':'MultiClass','bootstrap_type':'Bernoulli'})
    
    ctb.fit(X_train, y_train, eval_set=(X_valid, y_valid), verbose=0)
    
    ctb_preds = ctb.predict_proba(X_valid)[:,:]
    ctb_test_preds = ctb.predict_proba(test)[:,:]
    
    print(f"Auc is{roc_auc_score(y_valid, ctb_preds, multi_class='ovr',average='macro')}")
    
    lgb = LGBMClassifier(n_estimators=10000, 
                         early_stopping_rounds=100, 
                         **{'max_depth': 7, 
                            'subsample': 0.5528027451731908, 
                            'colsample_bytree': 0.26714040676512085, 
                            'learning_rate': 0.11430358849370452,
                            'objective':'multiclass','num_classes':6,'n_jobs':-1})
    
    lgb.fit(X_train, y_train, eval_set=(X_valid, y_valid), verbose=0)
    
    lgb_preds = lgb.predict_proba(X_valid)[:,:]
    lgb_test_preds = lgb.predict_proba(test)[:,:]
    
    print(f"Auc is : {roc_auc_score(y_valid, lgb_preds, multi_class='ovr',average='macro')}")
    
    xgb = XGBClassifier(n_estimators=10000, 
                         early_stopping_rounds=100,
                        **{'max_depth': 12, 'subsample': 0.5549531235327668, 'colsample_bytree': 0.20335345531973037, 
                           'learning_rate': 0.027251718255125106,'objective':'multiclass','n_jobs':-1})
    
    xgb.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], verbose=0)
    
    xgb_preds = xgb.predict_proba(X_valid)[:,:]
    xgb_test_preds = xgb.predict_proba(test)[:,:]
    
    print(f"Auc is : {roc_auc_score(y_valid, xgb_preds, multi_class='ovr',average='macro')}")
            
    meta_train = [lgb_preds, ctb_preds, xgb_preds]
    meta_test = [lgb_test_preds, ctb_test_preds, xgb_test_preds]

    final_train.append(meta_train)
    final_test.append(meta_test)
    #还有问题待修改
    def roc_auc(weights):
        fpred = np.zeros(meta_train[0].shape)
        for i, pred in enumerate(meta_train):
            fpred += weights[i] * pred

        fpred = fpred / fpred.sum(axis=1, keepdims=True)

        return -roc_auc_score(y_valid, fpred, multi_class='ovr', average='macro')

    starting_values = [0.33]*len(meta_train) 
    cons = ({'type':'eq','fun':lambda w: 1-sum(w)})
    bounds = [(-1,1)]*len(meta_train) 
    res = minimize(roc_auc, starting_values, method='Nelder-Mead', bounds=bounds, constraints=cons)

    print(-res["fun"])
    
    cv -= res["fun"]
    
    for i, pred in enumerate(meta_test):
        test_preds += res["x"][i] * pred#简单的加总每次CV的结果

print(f"CV: {cv/(fold+1)}")

---------->Fold 1<----------
Auc is0.9858581487884291
[LightGBM] [Warning] early_stopping_round is set=100, early_stopping_rounds=100 will be ignored. Current value: early_stopping_round=100


d:\python\envs\automl\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


Auc is : 0.9871420798127959
Auc is : 0.9869624657267716


d:\python\envs\automl\lib\site-packages\scipy\optimize\_minimize.py:569: RuntimeWarning: Method Nelder-Mead cannot handle constraints.
  warn('Method %s cannot handle constraints.' % method,


0.9879390838702858
---------->Fold 2<----------
Auc is0.9851382544470413
[LightGBM] [Warning] early_stopping_round is set=100, early_stopping_rounds=100 will be ignored. Current value: early_stopping_round=100


...

0.9859446930923168
---------->Fold 5<----------
Auc is0.9852869706582711
[LightGBM] [Warning] early_stopping_round is set=100, early_stopping_rounds=100 will be ignored. Current value: early_stopping_round=100


d:\python\envs\automl\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


Auc is : 0.9869842368194174
Auc is : 0.9861161917137274


d:\python\envs\automl\lib\site-packages\scipy\optimize\_minimize.py:569: RuntimeWarning: Method Nelder-Mead cannot handle constraints.
  warn('Method %s cannot handle constraints.' % method,


0.9877006092632034
CV: 0.987884906722597


In [ ]:
for i in tqdm(range(len(feature))):
    for j in range(i + 1, len(feature)):
        ft1 = feature[i]
        ft2 = feature[j]
        df[ft1].astype(str) + '_' + df[ft2].astype(str)
        for k in range(j + 1, len(feature)):
            ft3 = feature[k]
            df[ft1].astype(str) + '_' + df[ft2].astype(str) + '_' + df[ft3].astype(str)

In [ ]:
new_columns_df = {}
for i in tqdm(feature):
    if i not in ['casulaties_vechicles','Number_of_Casualties','Number_of_Vehicles']:
        for j in ['Number_of_Vehicles','Number_of_Casualties','casulaties_vechicles']:
            if j != 'casulaties_vechicles':
            #创建统计特征
                tmp_dict = df.groupby([i])[j].agg(['mean','median','max','sum']).to_dict()
                for k in tmp_dict:
                    calcu_feature.append(f"{k}_{j}_by_{i}")
                    new_columns_df[f"{k}_{j}_by_{i}"] = df[i].map(tmp_dict[k]).astype('float32')
            else:
                tmp_dict = df.groupby([i])[j].agg(['mean','median']).to_dict()
                for k in tmp_dict:
                    calcu_feature.append(f"{k}_{j}_by_{i}")
                    new_columns_df[f"{k}_{j}_by_{i}"] = df[i].map(tmp_dict[k]).astype('float32')

In [33]:
re = np.argmax(test_preds,axis=1)

In [35]:
import json
predictions_dict = {str(i):int(re[i]) for i in range(len(re))}
with open("submit.json", "w") as outfile:
    json.dump(predictions_dict, outfile)